In [ ]:
import os
import math
import json
import pickle
import keras
import random
import numpy as np
import tensorflow as tf
from tensorflow.data import Dataset

from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from keras.optimizers import RMSprop, SGD, Adam
from keras.applications import MobileNet, ResNet50, InceptionV3
from keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from keras.applications.resnet50 import preprocess_input as resnet_preprocess
from keras.applications.inception_v3 import preprocess_input as inception_preprocess
from keras.regularizers import l2
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model
from keras.callbacks import Callback, LearningRateScheduler
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv1D, Attention, GlobalAveragePooling1D, BatchNormalization, Layer
from keras_facenet import FaceNet

random.seed(123)
tf.random.set_seed(12)
np.random.seed(123)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
input_shape = (224, 224, 3)
train_path = './data/train'

# Training pairs generating

Available training pairs from csv files are splitted to train - validation sets. Those pairs are positive(there is blood relation). For each set(train/valid) we additionally generate negative pairs.

Positive pairs are generated according to the input csv file. For each person of positive pair we create one negative pair.
In total we'll have twice more negative than positive pairs.

In [ ]:
def make_image_pair(pair, input_shape, shuffle=True, slice_imgs=1):
    '''
    Create pair of embeddings.
    
    Arguments:
    p1, p2 -- paths to persons' images directories (familyID/personID)
    
    Returns:
    pairs -- array of image pairs, pairing is alligned to smaller number of images
    ''' 
    p1, p2 = [os.path.join(train_path, p) for p in pair]
    
    p1_imgs = os.listdir(p1)
    p2_imgs = os.listdir(p2)
    
    if shuffle:
        random.shuffle(p1_imgs)
        random.shuffle(p2_imgs)
    
    p1_imgs = p1_imgs[:slice_imgs]
    p2_imgs = p2_imgs[:slice_imgs]
    
    for i in range(len(p1_imgs)):
        for j in range(len(p2_imgs)):
            img1_path = os.path.join(p1, p1_imgs[i])
            img2_path = os.path.join(p2, p2_imgs[j])
            img1 = image.load_img(img1_path, target_size=(input_shape[0], input_shape[1]))
            img2 = image.load_img(img2_path, target_size=(input_shape[0], input_shape[1]))
            img1 = np.array(img1).astype('float32')
            img2 = np.array(img2).astype('float32')
            
            yield img1, img2

In [ ]:
def pairs_set(input_pairs, input_shape, shuffle=True, slice_imgs=1):
    for pair, label in input_pairs:
        try:
            emb_pairs = make_image_pair(pair, input_shape, shuffle, slice_imgs)
            for emb_pair in emb_pairs:
                yield emb_pair, label
        except (KeyError, FileNotFoundError):
            continue

def batched_pairs(input_pairs, batch_size, dataset_period, input_shape, preprocess, shuffle=True, slice_imgs=1):
    imgs1 = []
    imgs2 = []
    labels = []
    counter = 0
    for example in pairs_set(input_pairs, input_shape, shuffle, slice_imgs):
        # Get every nth sample
        counter += 1
        if counter % dataset_period:
            continue
        
        exmpls, label = example
        exmpl1, exmpl2 = exmpls
        imgs1.append(exmpl1)
        imgs2.append(exmpl2)
        labels.append(label)
        if len(labels) == batch_size:
            yield {'input_1:0':preprocess(np.array(imgs1)), 'input_2:0':preprocess(np.array(imgs2))}, np.array(labels).astype(float)
            imgs1, imgs2, labels = [], [], []

In [ ]:
with open('train_val_set.json', 'r') as f:
    train_val_set = json.load(f)

train_rlt_list, neg_train_rltshps, valid_rlt_list, neg_valid_rltshps = list(train_val_set.values())
train_rlt_list = train_rlt_list * 4

train_rlts = list(zip(train_rlt_list + neg_train_rltshps, [True]*len(train_rlt_list) + [False]*len(neg_train_rltshps)))
val_rlts = list(zip(valid_rlt_list + neg_valid_rltshps, [True]*len(valid_rlt_list) + [False]*len(neg_valid_rltshps)))

random.shuffle(train_rlts)
random.shuffle(val_rlts)

# Siamese network

Initial experimenting is done with conv1D deep neural network, as additional option for experimenting there is simple attention module.

In [ ]:
def mobilenet(input_shape, l2_value, dropout):
    mobile = MobileNet(
        input_shape=input_shape,
        dropout=dropout,
        include_top=False,
        pooling='avg',
        alpha=.75,
        weights='imagenet'
    )
    
    for layer in mobile.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(32, kernel_regularizer=l2(l2_value), activation='relu')(mobile.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(mobile.input, x)

def inception(input_shape, l2_value, dropout):
    inception = InceptionV3(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    for layer in inception.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(64, kernel_regularizer=l2(l2_value), activation='relu')(inception.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(inception.input, x)

def resnet50(input_shape, l2_value, dropout):
    resnet = ResNet50(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    for layer in resnet.layers:
        layer.trainable = True
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(32, kernel_regularizer=l2(l2_value), activation='relu')(resnet.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(resnet.input, x)

# Loss and metrics functions

In [ ]:
MARGIN = 1 - math.sqrt(3)/2

def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def cosine_distance(vectors):
    x, y = vectors
    x_norm = tf.norm(x, axis=1)
    y_norm = tf.norm(y, axis=1)
    x_y_dot = tf.einsum('ij,ij->i', x, y)
    cos_sim = x_y_dot / (x_norm * y_norm + K.epsilon())
    return 1. - cos_sim

def cos_euc_dist(vectors):
    euc = euclidean_distance(vectors)
    cos_dist = cosine_distance(vectors)
    return (1. - cos_dist) * euc

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''
    Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    weight_pos = 1. / (K.sum(y_true) + K.epsilon())
    weight_neg = 1. / (K.sum(1. - y_true) + K.epsilon())
    
    # Margins for positive and negative pairs in the batch
    margin_pos = 0.80 * MARGIN
    margin_neg = 1.2 * MARGIN
    
    pos_pred = y_true * y_pred
    pos_neg = (1 - y_true) * y_pred
    
    pred_pos_m = K.mean(pos_pred, axis=0)
    pred_neg_m = K.mean(pos_neg, axis=0)
    
    variance_loss = weight_pos * K.mean(K.square(pred_pos_m - pos_pred)) + weight_neg * K.mean(K.square(pred_neg_m - pos_neg))
    dist_diff = K.square((margin_neg - margin_pos) - (pred_neg_m - pred_pos_m))
    
    square_pred = K.square(K.maximum(y_pred - margin_pos, 0))
    square_neg = K.square(K.maximum(margin_neg - y_pred, 0))
    return K.mean(y_true * weight_pos * square_pred + (1 - y_true) * weight_neg * square_neg) + 0.6 * variance_loss + 0.1 * dist_diff

# Run training

In [ ]:
learning_rate = 8e-7
l2_value = 1e-7
dropout = 0.1
epochs = 1000
batch_size = 36
eval_batch_size = 128
dataset_period = 1
eval_dataset_period = 1
model_type = 'inception'
preprocess = inception_preprocess
# 'euclidian' or 'cosine_distance'
distance_type = 'cosine_distance'
optimizer = 'Adam'

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch == 10:
        return 0.5 * lr
    elif epoch == 100:
        return 0.3 * lr
    elif epoch == 150:
        return 0.5 * lr
    elif epoch == 200:
        return 0.7 * lr
    elif epoch == 300:
        return 0.3 *lr
    elif epoch == 500:
        return 0.6 *lr
    return lr
    
lr_callback = LearningRateScheduler(scheduler)

# Create dictionary of parameters for saving configuration
train_config = {}
for name in [
    'learning_rate',
    'l2_value',
    'dropout',
    'epochs',
    'batch_size',
    'model_type',
    'dataset_period',
    'eval_dataset_period',
    'distance_type',
    'optimizer',
    'MARGIN'
]:
    train_config[name] = eval(name)

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2),
    keras.layers.experimental.preprocessing.RandomContrast(factor=0.2),
    keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2)
])

In [ ]:
base_network = eval(model_type)(input_shape, l2_value, dropout)
base_network.count_params()

In [ ]:
# Creation of Siamese network
input1 = Input(shape=input_shape)
input2 = Input(shape=input_shape)

processed1 = base_network(data_augmentation(input1))
processed2 = base_network(data_augmentation(input2))

In [ ]:
dist_function = eval(distance_type)
distance = Lambda(dist_function,
                  output_shape=eucl_dist_output_shape)([processed1, processed2])

model = Model([input1, input2], distance)
optimizer = eval(optimizer)(learning_rate=learning_rate)
model.compile(loss=contrastive_loss, optimizer=optimizer)

## Run tensorboard plugin in order to track changes of training

In [ ]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=./logs --port=7007

## Training Callbacks

In [ ]:
# Get training set length
train_len = 0
for pair, label in train_rlts:
    try:
        p1, p2 = [os.path.join(train_path, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        train_len += 1
    except FileNotFoundError:
        continue

train_len = train_len // dataset_period

val_len = 0
for pair, label in val_rlts:
    try:
        p1, p2 = [os.path.join(train_path, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        val_len += 2*2
    except FileNotFoundError:
        continue

val_len = val_len // eval_dataset_period

print(f'Train set length: {train_len}')
print(f'Valid set length: {val_len}')

In [ ]:
def val_distance_stats(predictions, labels):
    val_pos = predictions[labels.astype(np.bool)]
    val_neg = predictions[(1 - labels).astype(np.bool)]
    val_pos_m, val_pos_s = np.mean(val_pos), np.std(val_pos)
    val_neg_m, val_neg_s = np.mean(val_neg), np.std(val_neg)
    
    return val_pos_m, val_pos_s, val_neg_m, val_neg_s
  
class MetricCallback(keras.callbacks.Callback):
    def __init__(self, logdir):
        super(Callback, self).__init__()
        if not os.path.exists(logdir):
            os.makedirs(logdir)
        self.train_writer = tf.summary.create_file_writer(logdir + '/train')
        self.valid_writer = tf.summary.create_file_writer(logdir + '/valid')
        self.class_encoded = {
            0: 'not_related',
            1: 'related'
        }
        
    def tb_writer(self, items_to_write, wtype, epoch):
        writer = self.train_writer if wtype == 'train' else self.valid_writer
        
        with writer.as_default():
            for name, value in items_to_write.items():
                tf.summary.scalar(name, value, epoch)
            writer.flush()
        
    def on_epoch_end(self, epoch, logs={}):
        val_true = []
        val_pred = []
        for batch in batched_pairs(val_rlts, eval_batch_size, eval_dataset_period, input_shape, preprocess, shuffle=False, slice_imgs=2):
            val_pred.append(self.model.predict(batch))
            val_true.extend(list(batch[1]))
        
        val_true = np.array(val_true)
        val_pred = np.concatenate(val_pred, axis=0)
        val_loss = contrastive_loss(K.constant(val_true), K.constant(val_pred))
        
        val_true = val_true.astype(int)
        val_pos_m, val_pos_s, val_neg_m, val_neg_s = val_distance_stats(val_pred, val_true)
#         threshold = (val_pos_m + val_neg_m ) / 2
        threshold = MARGIN
        
        # Precision and recall
        val_pred = (val_pred.squeeze() < threshold).astype(int)
        valid_precision, valid_recall, _, _ = precision_recall_fscore_support(val_true, val_pred, labels=[0, 1])
        valid_accuracy = accuracy_score(val_true, val_pred)
        
        train_loss = logs['loss']
        logs = {}
        logs['train/loss'] = train_loss
        
        self.tb_writer(logs, wtype='train', epoch=epoch)
        
        logs = {}
        logs['valid/loss'] = val_loss
        for k, v in self.class_encoded.items():
            logs['valid/precision/' + v] = valid_precision[k]
            logs['valid/recall/' + v] = valid_recall[k]
            logs['valid/dist_mean/' + v] = val_pos_m if k else val_neg_m
            logs['valid/dist_std/' + v] = val_pos_s if k else val_neg_s
        
        logs['valid/accuracy'] = valid_accuracy

        self.tb_writer(logs, wtype='valid', epoch=epoch)

In [ ]:
model_name = 'model_007_inception_cos'

#Save training configuration
with open(f'configs/{model_name}.json', 'w') as f:
    json.dump(train_config, f)

logdir = os.path.join('logs', model_name)
ckpt_dir = os.path.join('checkpoints', model_name)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
ckpt_callback = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, 'weights.{epoch:02d}.hdf5'),
    save_weights_only=False,
    period=3
)
metric_callback = MetricCallback(logdir)

In [ ]:
def repeat_generator(rlts, batch_size, dataset_period, input_shape, preprocess):
    while True:
        for e in batched_pairs(rlts, batch_size, dataset_period, input_shape, preprocess):
            yield e
            
model.fit(
    repeat_generator(train_rlts, batch_size, dataset_period, input_shape, preprocess),
    epochs=epochs,
    steps_per_epoch=500,
    callbacks=[metric_callback, ckpt_callback, lr_callback]
)

# Submission

In [ ]:
# Load submission pairs
submission_path = 'data/sample_submission.csv'
submission_df = pd.read_csv(submission_path)

In [ ]:
# Load models
ckpt_path = 'checkpoints/model_005_inception_cos/weights.16.hdf5'
model.load_weights(ckpt_path)

In [ ]:
# Get the threshold according to validation ds
threshold = MARGIN

# Iterate over submission pairs
is_related = submission_df['is_related']
predictions = []
for idx, row in submission_df.iterrows():
    # Load images
    img_pair = row['img_pair']
    img1_name, img2_name = img_pair.split('-')
    img1_path = os.path.join('data/test', img1_name)
    img2_path = os.path.join('data/test', img2_name)
    img1 = image.load_img(img1_path)
    img2 = image.load_img(img2_path)
    img1 = np.array(img1).astype('float32')
    img2 = np.array(img2).astype('float32')
    
    # Get FaceNet embeddings
    embedding1 = embedder.embeddings([img1])
    embedding2 = embedder.embeddings([img2])
    
    # Do an inference, if distance is smaller than threshold
    # then there is the relation
    y_pred = model.predict([preprocess(img1), preprocess(img2)])
    predictions.append(y_pred[0])
    if y_pred.squeeze() < threshold:
        is_related[idx] = 1
    
    # Print step
    if idx % 100 == 0:
        print(f'Processed rows: {idx}')
        
submission_df.to_csv(f'submission_{model_name}.csv', index=False)

In [ ]:
plt.hist(predictions, 20)
plt.show()

In [ ]:
thr = 0.85
for i, p in enumerate(predictions):
    if p < thr:
        is_related[i] = 1
    else:
        is_related[i] = 0
submission_df.to_csv(f'submission_test.csv', index=False)